# MONAI Auto3Dseg AutoRunner

In this notebook, An interface to run the Auto3Dseg pipeline in less than 5 lines of code

## 1. Set up environment, imports and datasets
### 1.1 Set up Environment

In [1]:
!python -c "import monai" || pip install -q "monai-weekly[nibabel]"

### 1.2 Set up imports

In [2]:
import os
from monai.bundle.config_parser import ConfigParser
from monai.apps import download_and_extract

from monai.apps.auto3dseg import AutoRunner

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 1.3 Download public datasets

In [3]:
root = "./"
msd_task = "Task05_Prostate"
resource = "https://msd-for-monai.s3-us-west-2.amazonaws.com/" + msd_task + ".tar"
compressed_file = os.path.join(root, msd_task + ".tar")
if os.path.exists(root):
    download_and_extract(resource, compressed_file, root)

dataroot = os.path.join(root, msd_task)
datalist = "../tasks/msd/Task05_Prostate/msd_task05_prostate_folds.json"

2022-09-15 13:48:20,142 - INFO - Expected md5 is None, skip md5 check for file Task05_Prostate.tar.
2022-09-15 13:48:20,142 - INFO - File exists: Task05_Prostate.tar, skipped downloading.
2022-09-15 13:48:20,143 - INFO - Non-empty folder exists in Task05_Prostate, skipped extracting.


### 1.4 Prepare a input YAML configuration

In [ ]:
data_src_cfg = {
    "name": "Task05_Prostate",
    "task": "segmentation",
    "modality": "MRI",
    "datalist": datalist,
    "dataroot": dataroot,
}
input = './input.yaml'
ConfigParser.export_config_file(data_src_cfg, input)

## 2. Run the Auto3Dseg pipeline in a few lines of code

Below is the typical usage of AutoRunner
```python
runner = AutoRunner(input=input)
runner.run()
```

The `run` command will take a long time because algos are being trained
Here we recommend the user to go through the usage of other commands below and run it in the end.

### 2.1 Use the default setting

In [ ]:
runner = AutoRunner(input=input)
# runner.run()

### 2.2 Use the dictionary instead of a YAML file as the input

In [ ]:
runner = AutoRunner(input=data_src_cfg)

## 3 Customize and configure the Auto3Dseg
### 3.1 Set your working directory

In [ ]:
runner = AutoRunner(work_dir='./my_workspace', input=input)

### 3.2 Use cached result to save computation time

AutoRunner saves intermediate results by default. The user can choose whether it uses the cached results or restart from scratch.

If the users want to start from scratch, they can set `not_use_cache` to True

In [ ]:
# This will restart from scratch and not use any cached results
runner = AutoRunner(input=input, not_use_cache=True)
# This will skip data analysis. If it was done before, AutoRunner will continue to the next step. If not, it will throw an error
runner = AutoRunner(input=input, analyze=False)

### 3.3 Output Ensemble Result

AutoRunner will perform inference on the testing data specified by the `datalist` in the data source config input. The inference result will be written to the `ensemble_output` folder under the working directory in the form of `nii.gz`. The user can choose the format by adding keyword arguments to the AutoRunner. A list of argument can be found in [MONAI tranforms documentation](https://docs.monai.io/en/stable/transforms.html#saveimage).

In [ ]:
runner = AutoRunner(input=input, output_dir='./output_dir')

## 4 Setting Auto3Dseg internal parameters
### 4.1 Change the number of folds for cross-validation

In [ ]:
runner = AutoRunner(input=input)
runner.set_num_fold(num_fold=2)

### 4.2 Customize traininig parameters by override the default values

In [ ]:
runner = AutoRunner(input=input)
# Note: among the provided bundles, most networks takes "num_iterations" to control the training iterations except segresnet
train_param = {"num_iterations": 8}
runner.set_training_params(params=train_param)

### 4.3 Customize the ensemble method (mean vs. majority voting)

In [ ]:
runner = AutoRunner(input=input)
runner.set_ensemble_method(ensemble_method_name="AlgoEnsembleBestByFold")

### 4.4 Customize the inference parameters by override the default values

In [ ]:
# set model ensemble method
pred_params = {
    'files_slices': slice(0, 2),  # only infer the first two files in the testing data
    'mode': "vote",              # use majority vote instead of mean to ensemble the predictions
    'sigmoid': True,             # when to use sigmoid to binarize the prediction and output the label
}
runner = AutoRunner(input=input)
runner.set_prediction_params(params=pred_params)

## 5 Train model with HPO (NNI Grid-search)
### 5.1 Apply HPO to search hyper-parameter in Auto3Dseg

Note: Auto3Dseg supports hyper parameter optimization (HPO) via NNI and Optuna backends. Notebook of how to use these modules can be found in this directory.
AutoRunner supports NNI backend with a grid search method via automatically generating a the NNI config and run `nnictl` commands in subprocess.
Note: to run the HPO, you need to ensure the development environment has `nni` package. Please refer to the [MONAI Installation Guide](https://docs.monai.io/en/stable/installation.html#installing-the-recommended-dependencies) for how to install the recommended dependencies.

In [ ]:
runner = AutoRunner(input=input, hpo=True)
search_space = {"learning_rate": {"_type": "choice", "_value": [0.0001, 0.001, 0.01, 0.1]}}
runner.set_nni_search_space(search_space)

### 5.2 Override the templated values

AutoRunner uses the following NNI config in its HPO module
```python
default_nni_config = {
    "trialCodeDirectory": ".",
    "trialGpuNumber": torch.cuda.device_count(),
    "trialConcurrency": 1,
    "maxTrialNumber": 10,
    "maxExperimentDuration": "1h",
    "tuner": {"name": "GridSearch"},
    "trainingService": {"platform": "local", "useActiveGpu": True},
}
```

It can be override by setting the hpo parameters

In [ ]:
runner = AutoRunner(input=input, hpo=True)
hpo_params = {"maxTrialNumber": 20}
search_space = {"learning_rate": {"_type": "choice", "_value": [0.0001, 0.001, 0.01, 0.1]}}
runner.set_hpo_params(params=hpo_params)
runner.set_nni_search_space(search_space)